In [32]:
import pandas as pd
import pymysql

In [33]:
conn = pymysql.connect(host = 'localhost',
                       port=3306,
                       user = 'root',
                       password = 'rootpass',
                       db = 'jeju')

In [34]:
sql_input = "SELECT * FROM jeju_data_web where contentscd = '관광지'"
sql_df = pd.read_sql_query(sql_input, conn)

C:\Users\TJ\.conda\envs\py38\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
drop_lsts = ["isSite","type","rCode","posExact","address","roadAddr","isCallLink","adult","endPageUrl","mobileEndPageUrl","ktCallMd","ppc","imageModDate","displayCategory","category","categoryPaths","previewImages","streetPanorama","skyPanorama","insidePanorama","interiorPanorama","indoorPanorama","poiInfo","entranceCoords","theme","petrolInfo","marker","markerSelected","datalab","dynamicData","markerLabel","isParkingSupported"]

In [4]:
raw_df=pd.read_csv('../naver_crawling/tour_data_raw.csv')

In [5]:
raw_df.drop(drop_lsts,axis=1,inplace=True)

,id,name,x,y,fullAddress,addressAbbr,fullRoadAddress,phone,description,way,...,bizhourInfo,options,menus,menuImages,hasNaverBooking,naverBookingUrl,naverbookingId,michelinGuide,broadcastInfo,reviewCount


In [11]:
images_df = raw_df[['id','name','urlList','images','imageURL','categories']]

In [17]:
images_df[images_df['imageURL'].isnull()]

,id,name,urlList,images,imageURL,categories
4,1033492321,강병대교회,[],[],NaN,"['종교', '개신교']"
7,17087735,개오름,[],[],NaN,"['여행,명소', '오름']"
8,16994185,성진이오름,[],[],NaN,"['여행,명소', '오름']"
9,17084701,무악,[],[],NaN,"['여행,명소', '오름']"
10,17084537,무악,[],[],NaN,"['지명', '봉우리,고지']"
...,...,...,...,...,...,...
1520,31348134,수영여,[],[],NaN,"['지명', '바위']"
1521,17040977,묵리고개,[],[],NaN,"['지명', '골짜기,고개']"
1522,38158121,뒷골,[],[],NaN,"['지명', '골짜기,고개']"
1523,38158123,금파골,[],[],NaN,"['지명', '골짜기,고개']"


In [19]:
df =pd.read_csv('../jeju_tour_places_modified.csv')

In [26]:
df.columns

Index(['alltag', 'contentsid', 'contentscd', 'title', 'address', 'roadaddress',
       'tag', 'introduction', 'readcnt', 'likecnt', 'reviewcnt', 'markcnt',
       'snssharecnt', 'schedulecnt', 'visitcnt', 'evelpt', 'latitude',
       'longitude', 'phoneno', 'sbst', 'PHOTOSHARECOUNT', 'cate2cd', 'cate3cd',
       'imgpath', 'thumbnailpath'],
      dtype='object')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   alltag           1103 non-null   object 
 1   contentsid       1103 non-null   object 
 2   contentscd       1103 non-null   object 
 3   title            1103 non-null   object 
 4   address          1099 non-null   object 
 5   roadaddress      1095 non-null   object 
 6   tag              1103 non-null   object 
 7   introduction     1101 non-null   object 
 8   readcnt          1103 non-null   int64  
 9   likecnt          1103 non-null   int64  
 10  reviewcnt        1103 non-null   int64  
 11  markcnt          1103 non-null   int64  
 12  snssharecnt      1103 non-null   int64  
 13  schedulecnt      1103 non-null   int64  
 14  visitcnt         1103 non-null   int64  
 15  evelpt           793 non-null    float64
 16  latitude         1094 non-null   float64
 17  longitude     

In [109]:
df[df['title'].str.startswith(sql_df['name'][0][:-6])].loc[:,'contentsid'].values

array(['CONT_000000000500457'], dtype=object)

In [110]:
def find_contentsid(name):
    word_length = len(name)
    for i in range(1,word_length):
        search_word = name[:-i]
        result = df[df['title'].str.startswith(search_word)].loc[:,'contentsid'].values
        if len(result) == 0:
            continue
        elif len(result) == 1:
            print(f'{name}의 contentsid발견')
            return result
        else:
            print(f'{name}의 contesid확인 필요')
            return result
            

In [111]:
testr=find_contentsid(sql_df['name'][0])

오설록 티 뮤지엄의 contentsid발견


In [112]:
testr

array(['CONT_000000000500457'], dtype=object)

In [104]:
sql_df['name'][0]

'오설록 티 뮤지엄'

In [114]:
namelsts = sql_df['name']

In [118]:
contentsid_lsts = []
for name in namelsts:
    tmp_dict={'name':name}
    tmp_result=find_contentsid(name)
    if len(tmp_result) == 1:
        tmp_dict['contentsid']=tmp_result
    else:
        tmp_dict['needcheck'] = True
        tmp_dict['contentsid'] = tmp_result
    contentsid_lsts.append(tmp_dict)
    print(f'{name} 검색 완료')

오설록 티 뮤지엄의 contentsid발견
오설록 티 뮤지엄 검색 완료
휴애리자연생활공원의 contentsid발견
휴애리자연생활공원 검색 완료
새연교의 contentsid발견
새연교 검색 완료
정방폭포의 contentsid발견
정방폭포 검색 완료
만장굴의 contentsid발견
만장굴 검색 완료
한림공원의 contentsid발견
한림공원 검색 완료
에코랜드 테마파크의 contentsid발견
에코랜드 테마파크 검색 완료
노형수퍼마켙의 contentsid발견
노형수퍼마켙 검색 완료
세계자동차&피아노 박물관의 contentsid발견
세계자동차&피아노 박물관 검색 완료
9.81 파크의 contentsid발견
9.81 파크 검색 완료
산굼부리의 contentsid발견
산굼부리 검색 완료
산방산의 contesid확인 필요
산방산 검색 완료
스누피가든의 contentsid발견
스누피가든 검색 완료
카멜리아힐의 contentsid발견
카멜리아힐 검색 완료
김녕해수욕장의 contesid확인 필요
김녕해수욕장 검색 완료
월정리해수욕장의 contesid확인 필요
월정리해수욕장 검색 완료
함덕해수욕장의 contentsid발견
함덕해수욕장 검색 완료
세화해수욕장의 contesid확인 필요
세화해수욕장 검색 완료
용두암의 contesid확인 필요
용두암 검색 완료
도두동무지개해안도로의 contentsid발견
도두동무지개해안도로 검색 완료
우도의 contesid확인 필요
우도 검색 완료
이호테우해수욕장의 contentsid발견
이호테우해수욕장 검색 완료
성산항의 contesid확인 필요
성산항 검색 완료
하귀애월해안도로의 contesid확인 필요
하귀애월해안도로 검색 완료
한라수목원의 contentsid발견
한라수목원 검색 완료
용눈이오름의 contentsid발견
용눈이오름 검색 완료
성산일출봉의 contentsid발견
성산일출봉 검색 완료
광치기해변의 contentsid발견
광치기해변 검색 완료
곽지해수욕장의 contesid확인 필요
곽지해수욕장 검색 완료
섭지코지의 contentsi

In [120]:
contentsid_df=pd.DataFrame(contentsid_lsts)

In [126]:
contentsid_df['needcheck'].fillna(False,inplace=True)

In [128]:
contentsid_df[contentsid_df['needcheck']]

,name,contentsid,needcheck
11,산방산,"[CNTS_000000000022459, CONT_000000000500285, C...",True
14,김녕해수욕장,"[CONT_000000000500083, CNTS_000000000019768]",True
15,월정리해수욕장,"[CONT_000000000500496, CNTS_000000000022516]",True
17,세화해수욕장,"[CONT_000000000500362, CONT_000000000500361]",True
18,용두암,"[CONT_000000000500469, CNTS_000000000019909]",True
20,우도,"[CONT_000000000500476, CONT_000000000500477, C...",True
22,성산항,"[CNTS_000000000000978, CONT_000000000500349, C...",True
23,하귀애월해안도로,"[CNTS_000000000000971, CNTS_000000000022753, C...",True
28,곽지해수욕장,"[CNTS_000000000019761, CONT_000000000500056]",True
34,한라산,"[CONT_000000000500685, CNTS_200000000009551, C...",True


In [157]:
df[df['title'].str.startswith('산방산')]

,alltag,contentsid,contentscd,title,address,roadaddress,tag,introduction,readcnt,likecnt,...,evelpt,latitude,longitude,phoneno,sbst,PHOTOSHARECOUNT,cate2cd,cate3cd,imgpath,thumbnailpath
11,"산,걷기/등산,경관/포토,친구,커플,맑음,여름",CNTS_000000000022459,관광지,산방산 수국,제주특별자치도 서귀포시 안덕면 산방로,제주특별자치도 서귀포시 안덕면 산방로,"산,걷기/등산,경관/포토,친구,커플,맑음,여름,자연경관,포토스팟,어트랙션,수국",산방산 근처의 수국길.,4067,0,...,5.0,33.238625,126.314610,--,수국이 활짝 피어있는 포토존.\r\n산방산 가는길에 길게 늘어서 있는 수국 꽃길이다...,NaN,자연,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
262,"지질트레일,산,걷기/등산,맑음,가을,공용주차장,장애인 화장실,장애인 전용 주차장,아...",CONT_000000000500285,관광지,산방산,제주특별자치도 서귀포시 안덕면 사계리 산 16,제주특별자치도 서귀포시 안덕면 사계리 산 16,"지질트레일,산,걷기/등산,맑음,가을,자연경관,봄꽃,유채꽃,유네스코",설문대할망이 한라산 꼭대기를 뽑아 던져놓았다는 산방산 \r\n· 명승 제77호\r\...,60491,11,...,5.0,33.236614,126.312890,064-794-2940,"제주 서남부 지역을 달리다 보면, 안덕면 사계리 랜드마크인 산방산을 볼 수 있는데,...",165.0,자연,산,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
463,"지질트레일,걷기/등산,친구,커플,맑음,봄,공용주차장,화장실,편의점,유도 및 안내시설...",CNTS_000000000021380,관광지,산방산 용머리해안 지질트레일,제주특별자치도 서귀포시 안덕면 사계리 981,제주특별자치도 서귀포시 안덕면 사계북로41번길 192,"지질트레일,걷기/등산,친구,커플,맑음,봄,자연경관,체험,언택트,유네스코,어트랙션,봄...",80만년 지구의 시간과 함께 용머리해안과 산방산을 품은 길,25197,9,...,5.0,33.235634,126.309460,064-792-3363,?한국의 아름다운 길로 뽑혔을만큼 놀라운 경치를 뽐내는 형제해안로가 포함된 산방산 ...,64.0,도보,지질트레일,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
699,"액티비티,친구,커플,공용주차장,현금결제,카드결제,화장실,무료 WIFI,편의점,음료대...",CONT_000000000500286,관광지,산방산 랜드,제주특별자치도 서귀포시 안덕면 사계남로216번길 24-32,제주특별자치도 서귀포시 안덕면 사계남로216번길 24-32,"액티비티,친구,커플,체험,레저/체험,어트랙션",웅장하고 신비로운 산방산을 배경으로 우리나라에서 가장 높게 올라가는 짜릿한 바이킹을...,9968,0,...,NaN,33.233696,126.311775,064-794-1425,산방산과 용머리 해안관광지구에 설치된 산방산랜드는 수학여행온 학생과 청소년들에게 인...,NaN,레저/체험,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
867,"실내,휴식/힐링,부모,아이,흐림,겨울",CONT_000000000500288,관광지,산방산탄산온천,제주특별자치도 서귀포시 안덕면 사계북로 41번길 192,제주특별자치도 서귀포시 안덕면 사계북로 41번길 192,"실내,휴식/힐링,부모,아이,흐림,겨울,자연경관,실내관광지,어린이,어트랙션","제주 유일, 국내 최고의 탄산온천, 제주 산방산탄산온천",72268,25,...,5.0,33.248665,126.298960,064-792-8300,"-온천수ph(6.58), 중성\r\n-온도 : 지하 600m기준 공저온도 32.55...",4.0,레저/체험,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
